In [6]:
import pandas as pd
import io
import requests
from IPython.core.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
%load_ext rpy2.ipython

pd.options.display.max_columns = 50

Tests covered in Chatper 9:
1. Sign Test
2. Wilcoxon Signed-Rank Test
3. Wilcoxon Rank-Sum Test

**Non-Parametric** tests are useful when either:
1. The data is ordinal or
2. The data is cardinal but the underlying assumptions for t-tests (normality of data or large sample data) are not met

# Data Types

**Cardinal data** are on a scale where it is meaningful to measure the distance between possible data values.

    Ex1. Body weight is a cardinal variable because a difference of 6 lb is twice as large as a difference of 3 lb.

For cardinal data, if the zero point is arbitrary, then the data are on an **interval scale**; if the zero point is fixed, then the data are on a **ratio scale**.

    Ex1. Body temperature is on an interval scale because the zero point is arbitrary. For example, the zero point has a different meaning for temperatures measured in Fahrenheit vs. Celsius.
    
    Ex2. Blood pressure and body weight are on ratio scales because the zero point is well defined in both instances.
    
**Ordinal data** can be ordered but do not have specific numeric values. Thus, common arithmetic cannot be performed on ordinal data in a meaningful way.

    Ex1. Visual acuity can be measured on an ordinal scale because we know 20–20 vision is better than 20–30, which is better than 20–40, and so on. How- ever, a numeric value cannot easily be assigned to each level of visual acuity that all ophthalmologists would agree on.
    
Data are on a **nominal scale** if different data values can be classified into categories but the categories have no specific ordering.

    Ex1. In classifying cause of death among patients with documented anal- gesic abuse, the following categories were used: (1) cardiovascular disease, (2) cancer, (3) renal or urogenital disease, and (4) all other causes of death. Cause of death is a good example of a nominal scale because the values (the categories of death) have no specific order with respect to each other.

# Sign Test

**Use Case**:
The sign test can be used when you want to perform hypothesis testing using ordinal data that has 3 states. For example, if we wanted to compare video games high scores for two people, A & B, where person A's high score is either greater than, less than, or equal to person B's high score.

**In the sign test, we're basically taking two, paired samples (taking n people and observing the effect of applying a treatment) and re-casting that data into a single sample *of differences* (i.e. was there an improvement, deterioration, or no effect at all?).**

***Then we test to see if the difference is non-zero.***

Since we've recasted the problem as a single sample of differences by calculating a difference d<sub>i</sub> for each sample. We want to test whether the difference is positive or not, we get a binomial distributed random variable (0 = no difference, 1 = positive difference).

See https://en.wikipedia.org/wiki/Sign_test for more info.

**Assumptions:**
1. The differences d<sub>i</sub> are assumed to be independent.
2. Each difference d<sub>i</sub> comes from the same continuous population (i.e. the data is paired).
3. The values x<sub>i</sub> and y<sub>i</sub> represent are ordered (at least the ordinal scale), so the comparisons "greater than", "less than", and "equal to" are meaningful.
4. n >= 20 (where n is the # of non-zero d<sub>i</sub>'s)

**Hypotheses:**<br>
H0: ∆ = 0<br>
Ha: ∆ != 0<br>
*Where ∆ is the population median of d<sub>i</sub>, and d<sub>i</sub> is the sign of the difference in observations (x<sub>i</sub> - y<sub>i</sub>).*

and a p-value of < 0.05 means that the two samples come from different populations (with type I error threshold = 0.05)

alternatively:

Let p = Pr(X > Y), and then test the null hypothesis <br>
H0: p = 0.50 <br>
Ha: p != 0.50 

**Alternative tests**:<br>
If the data is continuous, then a t-test is more appropriate.<br>
If the data is ranked, a wilcoxon signed-rank test is more appropriate. 

**Mechanics**:<br>
1. Python:<br>
`binom_test(count, nobs, value, alternative='two-sided')`
2. R:<br>
`prop.test(count, nobs, value, alternative="two.sided", conf.level=0.95, correct=TRUE)`<br>

count = # of samples where d<sub>i</sub> > 0<br>
nobs = # of samples where d<sub>i</sub> != 0<br>
value = p-value that we're assuming under H0 = 0.5

In [ ]:
# from page 340 
count = 18
nobs = 40
value = 0.5

In [17]:
from statsmodels.stats.proportion import proportions_ztest
proportions_ztest(count, nobs, value, alternative='two-sided', prop_var=False)

(-0.635641726163728, 0.5250099664186034)

In [93]:
from scipy.stats import binom_test
pvalue = binom_test(count, nobs, value, alternative='two-sided')
print(pvalue)

0.6358280026288411


In [18]:
%%R -i count,nobs,value
prop.test(x=count, n=nobs, p=value, alternative="two.sided", conf.level=0.95, correct=TRUE)


	1-sample proportions test with continuity correction

data:  count out of nobs, null probability value
X-squared = 0.225, df = 1, p-value = 0.6353
alternative hypothesis: true p is not equal to 0.5
95 percent confidence interval:
 0.2960304 0.6134103
sample estimates:
   p 
0.45 



Not sure why I get different p-values from the statsmodels python implementation vs the scipy python implementation- I suspect it may be because the proportions_ztest from statsmodels does not have a continuity correction option built in. 

In either case, the non-significant results mean that the ointment makes no difference. 

# Wilcoxon Signed-Rank Test

The Wilcoxon Signed-Rank Test is a non-parametric hypothesis test that can test the magnitude of differences (and not just the sign, like in the Sign Test).

**Non-parametric analog to the t-Test for paired samples.**

The test is based on the sum of the ranks, or the rank sum (R<sub>1</sub>), for the group of people with positive d<sub>i</sub> (the rank sum for people for whom treatment A is worse than treatment B)

**Assumptions**:
1. n >= 16 (where n is the # of non-zero d<sub>i</sub>'s, as then the normal approximation can be used for the sampling distribution of R<sub>1</sub>)
2. Continuous distribution (no ties)
3. Symmetric distribution
4. No outliers & no heavy tails

From https://www.statisticssolutions.com/assumptions-of-the-wilcox-sign-test/:
Since the Wilcoxon signed rank test does not require multivariate normality or homoscedasticity it is more robust than the dependent samples t test.  Apart from the cases where the two samples are not normally distributed it is better to use the Wilcoxon Sign Test when the sample includes outliers or heavy tails, two effects that highly influence the t test.

**Hypothesis Testing**:<br>
H0: ∆ = 0<br>
Ha: ∆ != 0<br>
*Where ∆ is the population median of d<sub>i</sub>, and d<sub>i</sub> is the difference in observations (x<sub>i</sub> - y<sub>i</sub>).*

*More generally:<br>*
H0: the distribution of the differences d<sub>i</sub> are symmetric around 0<br>
Ha: the distribution of the differences d<sub>i</sub> are not symmetric around 0

**Alternative Tests**
1. Sign Test, if magnitude of differences it not a concern, however the sign test has less power than the Wilcoxon Signed-Rank test
2. For cardinal data that is normally distributed, the Paired t-test will have more power. 

**Mechanics**:
1. Python<br>
`scipy.stats.wilcoxon(x=d, y=None, zero_method='wilcox', correction=True, alternative='two-sided')`<br>
d = differences between the two groups.
2. R <br>
One set of paired differences: <br>
`wilcox.test(x, y = NULL, alternative = "two-sided", mu = 0, paired = FALSE, exact = NULL, correct = TRUE, conf.int = FALSE)`<br>
Two sets of paired scores:<br>
`wilcox.test(x, y, alternative = "two-sided", mu = 0, paired = TRUE, exact = NULL, correct = TRUE, conf.int = FALSE)`

***Note the difference is in the input arguments {x,y} and the {paired} argument.***

In [95]:
# from example 9.16 on page 351
d = [-8, -7, -7, -7, -6, -6, -5, -5, -4, -3, -3, -3, -3, -3, 
     -2,-2,-2,-2,-1,-1,-1,-1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3]
len(d)

40

In [50]:
from scipy.stats import wilcoxon
wilcoxon(d, y=None, zero_method='wilcox', correction=True, alternative='two-sided')

WilcoxonResult(statistic=248.0, pvalue=0.028692758272822194)

In [65]:
%%R -i d
wilcox.test(unlist(d), y=NULL,alternative="two.sided",mu=0, paired=FALSE, exact=NULL,correct=TRUE,conf.int=FALSE)


	Wilcoxon signed rank test with continuity correction

data:  unlist(d)
V = 248, p-value = 0.02869
alternative hypothesis: true location is not equal to 0



OK, this time R didn't work out for us, but we got the same answer as the text did using the python implementation. The results are significant at the 5% level, but is ointment A better or worse than ointment B?

In order to determine that, we can probably just look at the distribution of these differences, but I think technically we would need to compare the calculated rank sum (248.0) to the an expected rank sum under the null hypothesis. 

Under H0: <br>
E(R1)=n(n+1)/4<br>
Var(R1)=n(n+1)(2n+1)/24<br>

In [30]:
# so since n = 40, we have 
n = len(d)
print(n * (n+1) / 4)

410.0


248 < 410.

We calculated the differences d<sub>i</sub> by taking the result on arm A and subtracting the result on arm B for each paired sample. So under H0, we'd expect the differences to be centered around 0

The test statistic 248 is calculated by calculating the rank sum for only the positive differences (where ointment B performed better than ointment A). 

*See text for how to calculate rank sum but it's just the SUM(count[i] * average_rank[i], for each i where d[i] > 0)*

Ex: R1 =10(7.5)+6(19.5)+2(28.0)=75+117+56=248

**Since 248 < 410, ointment B performs worse than ointment A.**

# Wilcoxon Rank-Sum Test

**Nonparametric analog to t-Test for two independent samples.**

*Also sometimes known as the Mann-Whitney U test*

**Assumptions**:
1. This test should be used only if both n1 and n2 are at least 10
2. Finally, a necessary condition for the strict validity of the rank-sum test is that the underlying distributions being compared must be continuous. (However, McNeil has investigated the use of this test in comparing discrete distributions and has found only small losses in power when applying this test to grouped data from normal distributions, compared with the actual ungrouped observations from such distribu- tions [3].)

**Hypothesis Testing**:<br>
H0: F<sub>1</sub> = F<sub>2</sub> <br>
H1: F<sub>1</sub>(x) = F<sub>2</sub>(x+∆), where ∆ ≠ 0.<br>
F<sub>1</sub> = cumulative distribution function (c.d.f.) of visual acuity for the 1st group, F<sub>2</sub> = cumulative distribution function of visual acuity for the 2nd group, and ∆ is a location shift of the c.d.f. for the 2nd group relative to the 1st group. 

If ∆ > 0, then 1st group patients tend to have better visual acuity than 2nd group patients; if ∆ < 0, then 1st group patients tend to have worse visual acuity than 2nd group patients; if ∆ = 0, then 1st group patients have the same acuity distribution as 2nd group patients.

**Mechanics**:
1. Python<br>
    `mannwhitneyu(x, y, use_continuity=True, alternative='two-sided')`
2. R <br>
The scores for the two groups need to be stored in a single variable (e.g., z). The grouping variable, which identi- fies the groups for individual observations, is stored in a separate variable (e.g., g) and should have two possible values. The syntax is as follows for the large sample method:
    `wilcox.test(z ~ g, alternative = “two.sided”, mu = 0, paired = FALSE, exact = NULL, correct = TRUE, conf.int = FALSE)`<br><br>
Alternatively, we can store the values for one group in a variable called x and the values for the other group in a variable called y and use the syntax:<br>
    `wilcox.test(x, y, alternative = "two-sided", mu=0, paired = FALSE, exact = NULL, correct = TRUE, conf.int = FALSE)`

In [55]:
# save the frequency counts 
dom = [20,20,20,20,20, 25, 25, 25, 25, 25, 25, 25, 25, 30, 30, 30, 30, 30, 30, 40, 40, 40, 50, 50]
sl = [20, 25, 25, 25, 25, 25, 30, 30, 30, 30, 40, 40, 40, 40, 50,50,50,50,50,50,50,50,60,60,60,60,60, 70, 70, 80]

In [71]:
from scipy.stats import mannwhitneyu
mannwhitneyu(dom, sl, use_continuity=True, alternative='two-sided')

MannwhitneyuResult(statistic=150.5, pvalue=0.00021951326877097123)

In [84]:
%%R -i dom,sl
wilcox.test(unlist(dom), unlist(sl), alternative="two.sided", mu=0, paired = FALSE, exact = NULL, correct = TRUE, conf.int = FALSE)


	Wilcoxon rank sum test with continuity correction

data:  unlist(dom) and unlist(sl)
W = 150.5, p-value = 0.0002195
alternative hypothesis: true location shift is not equal to 0



We got a slightly difference answer than the texrbook does - probably because I messed up in the data input. 

I'd like to see how the p-value changes when I use a the wilcoxon rank sum test vs. the independent t-test. Since the non-parametric test has less statistical power, I would expect the p-value it produces to be higher than the p-value for the continuous version, given the same data. Let's test that...

In [92]:
# Mann-Whitney U test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
# seed the random number generator
seed(1)
# generate two independent samples
data1 = 5 * randn(100) + 50
data2 = 5 * randn(100) + 51
# compare samples
stat, p = mannwhitneyu(data1, data2)
print('Mann-Whitney: Statistics=%.3f, p=%.3f' % (stat, p))

from scipy.stats import ttest_ind
stat, p = ttest_ind(data1, data2)
print('t-Test (independent): Statistics=%.3f, p=%.3f' % (stat, p))

Mann-Whitney: Statistics=4025.000, p=0.009
t-Test (independent): Statistics=-2.262, p=0.025


Welp, that didn't work out how I thought it would...